In [5]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score,precision_score,recall_score,f1_score, roc_auc_score

In [6]:
df = pd.read_csv('../data/crawler/unified-events-statistics.csv')
df = df.replace(['BLUE: first_blood',
    'BLUE: dragon',
    'BLUE: herald',
    'BLUE: first_tower_top',
    'BLUE: first_tower_mid',
    'BLUE: first_tower_bot',
    'BLUE: second_tower_top',
    'BLUE: second_tower_mid',
    'BLUE: second_tower_bot',
    'BLUE: third_tower_top',
    'BLUE: third_tower_mid',
    'BLUE: third_tower_bot',
    'BLUE: inhibitor_top',
    'BLUE: inhibitor_mid',
    'BLUE: inhibitor_bot',
    'BLUE: baron',
    'BLUE: elder_dragon',
    'BLUE: nexus_tower',
    'BLUE: nexus',
    'RED: first_blood',
    'RED: dragon',
    'RED: herald',
    'RED: first_tower_top',
    'RED: first_tower_mid',
    'RED: first_tower_bot',
    'RED: second_tower_top',
    'RED: second_tower_mid',
    'RED: second_tower_bot',
    'RED: third_tower_top',
    'RED: third_tower_mid',
    'RED: third_tower_bot',
    'RED: inhibitor_top',
    'RED: inhibitor_mid',
    'RED: inhibitor_bot',
    'RED: baron',
    'RED: elder_dragon',
    'RED: nexus_tower',
    'RED: nexus'], range(38))
df.head()

,golId,game,event1,event2,event3,event4,event5,event6,event7,event8,...,redMidGP,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA,result
0,36864,ESPORTSTMNT02_2556988,19,1,2,1,3,24,2,23,...,5,0.20,4.00,0,0.0,0.00,0,0.0,0.0,1
1,36865,ESPORTSTMNT05_2520933,19,21,1,22,25,24,20,21,...,4,0.25,3.55,10,0.4,4.07,2,0.5,5.4,0
2,36866,ESPORTSTMNT02_2557390,19,2,20,24,2,4,20,3,...,1,0.00,20.00,0,0.0,0.00,4,1.0,2.8,0
3,36867,ESPORTSTMNT02_2557426,20,0,21,20,21,5,23,1,...,5,0.20,4.00,5,0.6,9.12,0,0.0,0.0,0
4,36868,ESPORTSTMNT02_2557463,19,1,2,1,2,4,20,24,...,5,0.00,2.35,2,0.0,24.00,0,0.0,0.0,0


In [7]:
y = df['result'].copy()
X = df.drop(['golId','result','game','event4','event5','event6','event7','event8','event9','event10','event11','event12','event13','event14','event15','event16','event17','event18','event19','event20','event21','event22','event23','event24','event25','event26','event27','event28','event29','event30','event31','event32','event33','event34','event35','event36','event37','event38','event39','event40','event41','event42','event43','event44','event45'],axis=1)
X.head()

,event1,event2,event3,blueTopGP,blueTopWR,blueTopKDA,blueJungleGP,blueJungleWR,blueJungleKDA,blueMidGP,...,redJungleKDA,redMidGP,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA
0,19,1,2,1,1.00,2.00,1,1.0,2.25,1,...,0.0,5,0.20,4.00,0,0.0,0.00,0,0.0,0.0
1,19,21,1,1,0.00,3.00,2,1.0,1.11,3,...,4.0,4,0.25,3.55,10,0.4,4.07,2,0.5,5.4
2,19,2,20,0,0.00,0.00,1,0.0,3.25,0,...,0.0,1,0.00,20.00,0,0.0,0.00,4,1.0,2.8
3,20,0,21,7,0.14,4.14,1,0.0,5.00,2,...,0.0,5,0.20,4.00,5,0.6,9.12,0,0.0,0.0
4,19,1,2,1,0.00,2.00,1,1.0,2.25,0,...,6.6,5,0.00,2.35,2,0.0,24.00,0,0.0,0.0


In [8]:
def preprocess_input(X,y):
    X = X.copy()
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,stratify=y)
    scaler = StandardScaler()   
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train,X_test,y_train,y_test,scaler
X_train,X_test,y_train,y_test,scaler = preprocess_input(X,y)

In [9]:
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

models = {
    'Logistic Regression': LogisticRegression(max_iter=50000),
    'Support Vector Machine (RBF Kernel)': SVC(C=100,gamma=0.001,kernel='rbf',max_iter=50000,probability=True),
    'Decission Tree': DecisionTreeClassifier(),
    'Adaboost': AdaBoostClassifier(),
    'Random Forest': RandomForestClassifier(max_depth=25,n_estimators=600,min_samples_split=2,min_samples_leaf=1),
    'Gradient Boosting Classifier': GradientBoostingClassifier(criterion='friedman_mse',learning_rate=0.025,loss='deviance',max_depth=4,max_features='log2',min_samples_leaf=8,min_samples_split=3, n_estimators=100,subsample=0.5),
    'KNN': KNeighborsClassifier(n_neighbors=3),
    'Gaussian NB': GaussianNB()
}

for name, model in models.items():
  model.fit(X_train,y_train)
  print(name + ' trained')

Logistic Regression trained
Support Vector Machine (RBF Kernel) trained
Decission Tree trained
Adaboost trained
Random Forest trained
Gradient Boosting Classifier trained
KNN trained
Gaussian NB trained


In [10]:
scores_list = []

for name,model in models.items():    
    scores_list.append({
    'model': name,
    'amount_events': '3',
    'Balanced Accuracy': balanced_accuracy_score(y_test,model.predict(X_test)),
    'Precision':  precision_score(y_test,model.predict(X_test)),
    'Recall': recall_score(y_test,model.predict(X_test)),
    'F1-Score': f1_score(y_test,model.predict(X_test)),
    'auc': roc_auc_score(y_test,model.predict_proba(X_test)[:,1])
    })
scores = pd.DataFrame(scores_list)

In [11]:
best_model = scores.sort_values(by='auc', ascending=False).iloc[0]['model']
scores.sort_values(by='auc', ascending=False)

,model,amount_events,Balanced Accuracy,Precision,Recall,F1-Score,auc
5,Gradient Boosting Classifier,3,0.657673,0.662366,0.675439,0.668838,0.705195
0,Logistic Regression,3,0.651547,0.664399,0.642544,0.653289,0.703585
1,Support Vector Machine (RBF Kernel),3,0.636739,0.646667,0.638158,0.642384,0.699514
3,Adaboost,3,0.644415,0.652079,0.653509,0.652793,0.691717
4,Random Forest,3,0.630804,0.637149,0.646930,0.642002,0.677410
7,Gaussian NB,3,0.613663,0.628770,0.594298,0.611048,0.661572
6,KNN,3,0.551243,0.560870,0.565789,0.563319,0.573022
2,Decission Tree,3,0.554835,0.565410,0.559211,0.562293,0.554835


In [12]:
header = ['model', 'amount_events', 'auc']
scores.to_csv('../data/models-analysis/models.csv', columns = header, mode='a',index=False,header=False)

In [13]:
import pickle

with open("../models/model-3.pkl", "wb") as f:
    pickle.dump(models[best_model], f)

with open("../scalers/scaler-3.pkl", "wb") as f:
    pickle.dump(scaler, f)